# CHIME/FRB VOEvent Handling Tutorial

### Andrew V. Zwaniga, on behalf of CHIME/FRB Collaboration


### Version 1.1 (September 2021)

**Version 1.0**
- Detection and subsequent VOEvent handling.

**Version 1.1**
- Added walkthrough for retraction VOEvent handling.

## Table of Contents

1. Introduction
2. Requirements
3. Additional Resources
4. Load VOEvent from File
5. Get `<VOEvent>` Content
6. Get `<Who>` Content
7. Get `<What>` Content
8. Get `<WhereWhen>` Content
- 8.1 Notes on Comparing with `<What>`
9. Get `<How>` Content
10. Get `<Why>` Content
- 10.1 The `importance` of a CHIME/FRB VOEvent
- 10.2 The `probability` of a CHIME/FRB VOEvent
11. Get `<Citations>` Content
12. Conclusion
13. Future Work
---

## 1. Introduction 

In this notebook we demonstrate loading and parsing CHIME/FRB VOEvents of the following types: 
- `detection`
- `subsequent`
- `retraction`

## 2. Requirements

- `voeventparse`

## 3. Additional Resources

- `voeventparse` is a convenience lirbary for parsing VOEvent XML objects and offers a tutorial along with extensive documentation here https://voevent-parse.readthedocs.io/en/latest/tutorial/01-parsing.html
- Navigate to https://chime-frb.ca/analysis to find a dedicated VOEvent page with further instructions for getting acquainted with the Service.

---

## 4. Load VOEvent from File

VOEvents can be saved to and read from disk as `.xml` files using `voeventparse`. The result of loading from file is a VOEvent XML object that behaves just like other XML objects that users may be familar with from the `lxml` library.

In [2]:
# Set up working directoryimport oscwd = os.getcwd()# Import convenience library for parsing VOEvent XMLs
import voeventparse as vp

filename = cwd + "../static/FRB-DETECTION-2021-08-19-21:50:28.900992UTC+0000_7f17d982b315.xml"

with open(filename, "rb") as f:
    # Load VOEvent XML from file
    voevent = vp.load(f)
    # Dump VOEvent XML to a string variable
    voevent_string = vp.prettystr(voevent)

---
## 5. Get `<VOEvent>` Content

The `<VOEvent>` section contains information regarding the XML schema of the document, but most importantly the IVORN of the VOEvent. Of secondary importance is the `role` of the VOEvent, the value of which depends on the type of alert. 

- `detection` and `subsequent` have `role="observation"`
- `update` and `retraction` have `role="utility"`

In [3]:
# Get the <VOEvent> section as string
voevent_section = voevent_string[
    voevent_string.find("<VOEvent"):voevent_string.find("<Who>")
]
print(voevent_section)

# Get the VOEvent IVORN from the <VOEvent> section
ivorn = voevent.attrib["ivorn"]
print(f"VOEvent IVORN:\n--> {ivorn}\n")

# Get the VOEvent role from the <VOEvent> section
role = voevent.attrib["role"]
print(f"VOEvent role:\n--> {role}\n")

<VOEvent xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.ivoa.net/xml/VOEvent/v2.0 http://www.ivoa.net/xml/VOEvent/VOEvent-v2.0.xsd" version="2.0" role="observation" ivorn="ivo://ca.chimenet.frb/FRB-DETECTION-#2021-08-19-21:50:28.900992UTC+0000_7f17d982b315">
  
VOEvent IVORN:
--> ivo://ca.chimenet.frb/FRB-DETECTION-#2021-08-19-21:50:28.900992UTC+0000_7f17d982b315

VOEvent role:
--> observation



---
## 6. Get `<Who>` Content

The `<Who>` section contains identifying features of the agent that published the VOEvent. In this case, it is the CHIME/FRB VOEvent Service, and the main contact plus a valid email address for reaching out with comments or questions is provided.

- `AuthorIVORN` is the identifying IVORN of the CHIME/FRB VOEvent Broker
- `Date` is the UTC timestamp when the VOEvent was published
- `contactEmail` can be reached for comments or questions regarding the VOEvent
- `contactName` should be addressed in email communications 
- `shortName` is the name of the VOEvent agent

In [4]:
who_section = voevent_string[
    voevent_string.find("<Who>"):voevent_string.find("</Who>") + len("</Who>")
]
print(who_section)

<Who>
    <Description>CHIME/FRB VOEvent Service</Description>
    <AuthorIVORN>ivo://ca.chimenet.frb/contact</AuthorIVORN>
    <Date>2021-08-19T21:50:28+00:00</Date>
    <Author>
      <contactEmail>andrew.zwaniga@mail.mcgill.ca</contactEmail>
      <contactName>Andrew Zwaniga</contactName>
      <shortName>CHIME/FRB VOEvent Service</shortName>
    </Author>
  </Who>


In [5]:
# Get Author IVORN
author_ivorn = voevent.Who.AuthorIVORN
print(f"Author IVORN:\n--> {author_ivorn}")

Author IVORN:
--> ivo://ca.chimenet.frb/contact


In [6]:
# Get time when VOEvent was published
published = voevent.Who.Date
print(f"VOEvent was published on:\n--> {published}")

VOEvent was published on:
--> 2021-08-19T21:50:28+00:00


In [7]:
# Get VOEvent author information
contact_email = voevent.Who.Author.contactEmail
print(f"Send comments and questions to:\n--> {contact_email}\n")

contact_name = voevent.Who.Author.contactName
print(f"Address comments and questions to:\n--> {contact_name}\n")

short_name = voevent.Who.Author.shortName
print(f"VOEvent was published by:\n--> {short_name}")

Send comments and questions to:
--> andrew.zwaniga@mail.mcgill.ca

Address comments and questions to:
--> Andrew Zwaniga

VOEvent was published by:
--> CHIME/FRB VOEvent Service


---
## 7. Get `<What>` Content

The `<What>` section contains the most scientifically and observationally important meta data. Meta data is organized into three `<Group>`s as `<Param>`s:
- `observatory parameters` for all CHIME telescope and FRB instrument specifics
- `event parameters` for all FRB event measurements
- `advanced parameters` for meta data that may be relevant to user-defined triggering criteria or follow-up analysis

Each `<Param>` has up to four attributes:
- `name` is the name for the parameter
- `value` is a `float` or `int` numerical quantity, or `str` word or phrase
- `unit` is a `str` word representing the SI unit of the `value`
- `ucd` is a `str` word following the IVOA standard for Unified Content Descriptors (UCDs)


In [8]:
# Print the entire <What> section to standard output

what_section = voevent_string[
    voevent_string.find("<What>"):voevent_string.find("</What>") + len("</What>")
]
print(what_section)

<What>
    <Group name="observatory parameters">
      <Param ucd="time.resolution" unit="ms" value="0.983" name="sampling_time">
        <Description>FRB search time resolution</Description>
      </Param>
      <Param ucd="instr.bandwidth" unit="MHz" value="400" name="bandwidth">
        <Description>CHIME telescope bandwidth</Description>
      </Param>
      <Param ucd="em.freq;instr" unit="MHz" value="600" name="centre_frequency">
        <Description>CHIME telescope central frequency</Description>
      </Param>
      <Param ucd="" unit="" value="2" name="npol">
        <Description>The CHIME telescope has dual-polarization feeds</Description>
      </Param>
      <Param ucd="" unit="" value="8" name="bits_per_sample">
        <Description>CHIME/FRB samples 16384 frequency channels at 0.983 ms cadence as 8-bit integers</Description>
      </Param>
      <Param ucd="phot.antennaTemp" unit="K" value="50" name="tsys">
        <Description>CHIME receiver noise temperature</Descriptio

In [9]:
# Get all <Param> in <What> grouped by their <Group> using convenience method
groups = vp.get_grouped_params(voevent)

# Print name of group and param for user to select from
for group in groups:
    header = f'<Group> = "{group}"'
    print("\n" + header + "\n" + "-"*len(header))
    for param in groups[group]:
        print(f'<Param> name = "{param}"')


<Group> = "observatory parameters"
----------------------------------
<Param> name = "sampling_time"
<Param> name = "bandwidth"
<Param> name = "centre_frequency"
<Param> name = "npol"
<Param> name = "bits_per_sample"
<Param> name = "tsys"
<Param> name = "backend"

<Group> = "event parameters"
----------------------------
<Param> name = "event_no"
<Param> name = "known_source_name"
<Param> name = "event_type"
<Param> name = "pipeline_name"
<Param> name = "dm"
<Param> name = "dm_error"
<Param> name = "timestamp_utc"
<Param> name = "timestamp_utc_error"
<Param> name = "snr"
<Param> name = "pos_error_semiminor_deg_95"
<Param> name = "pos_error_semimajor_deg_95"

<Group> = "advanced parameters"
-------------------------------
<Param> name = "dm_gal_ne_2001_max"
<Param> name = "dm_gal_ymw_2016_max"
<Param> name = "timestamp_utc_inf_freq"
<Param> name = "timestamp_utc_inf_freq_error"
<Param> name = "spectral_index"
<Param> name = "spectral_index_error"


In [10]:
 def get_description(voevent, name):
    """
    Get the <Description> attribute of a <Param>. 
    
    Search for parameter by its ``name``, in the <Groups>
    under the <What> section.
    """
    for i in range(len(voevent.What.Group)):
        # Iterate over groups
        g = voevent.What.Group[i]
        for j in range(len(g.Param)):
            # Iterate over params in this group
            p = g.Param[j]
            if str(p.attrib.get("name")) == name:
                try:
                    return str(p.Description)
                except Exception:
                    return None

In [11]:
# Interactive session: user chooses a <Param> to see its details

looping = True
while looping:
    try:
        # User selects <Group>
        select_group = input("Select <Group> by name: ")
        assert select_group in groups, f"Unknown group name: {select_group}\nOptions: {[_ for _ in groups]}"

        # User selects <Param>
        select_param = input("Select <Param> by name: ")
        assert select_param in groups[select_group], f"Unknown param name: {select_param}\nOptions: {[_ for _ in groups[select_group]]}"
    
        header = f"<Param> with name = '{select_param}'" 
        print("\n" + header + "\n" + "-"*len(header))
    
        details = groups[select_group][select_param]
        value = details["value"]
        unit = details["unit"]
        ucd = details["ucd"]
        description = get_description(voevent, select_param)
    
        print(f'value = "{value}"')
        print(f'unit = "{unit}"')
        print(f'ucd = "{ucd}"')
        print(f'<Description> = "{description}"')
    
        looping = (input("\n<Enter>: Continue\n<AnyKey> + <Enter>: Quit\n")) == ""
    except Exception as e:
        print(f"{e}.\nTry again!")
        continue

Select <Group> by name: 
Unknown group name: 
Options: ['observatory parameters', 'event parameters', 'advanced parameters'].
Try again!
Select <Group> by name: 
Unknown group name: 
Options: ['observatory parameters', 'event parameters', 'advanced parameters'].
Try again!
Select <Group> by name: 
Unknown group name: 
Options: ['observatory parameters', 'event parameters', 'advanced parameters'].
Try again!
Select <Group> by name: 
Unknown group name: 
Options: ['observatory parameters', 'event parameters', 'advanced parameters'].
Try again!
Select <Group> by name: 
Unknown group name: 
Options: ['observatory parameters', 'event parameters', 'advanced parameters'].
Try again!
Select <Group> by name: 
Unknown group name: 
Options: ['observatory parameters', 'event parameters', 'advanced parameters'].
Try again!
Select <Group> by name: 
Unknown group name: 
Options: ['observatory parameters', 'event parameters', 'advanced parameters'].
Try again!
Select <Group> by name: 
Unknown group na

---
## 8. Get `<WhereWhen>` Content

The `<WhereWhen>` section specifically reports the coordinates and their uncertainties (together giving the on-sky localization region) in a more elaborate format than is used for `<What>`.

For CHIME/FRB VOEvents, the right ascension (RA) and declination (Dec) are reported along with the real-time localization region, the UTC time of detection, and the astronomical coordinate system. This region is calculated during the L2-L3 stage of the real-time FRB pipeline, where it is represented as an on-sky ellipse in (RA, dec) coordinates. In the VOEvent `<WhereWhen>`, however, only the maximum of the semi-minor and semi-major axes of said ellipse are reported. (Note that both axes are reported in `<What>` for convenience, as shown below.)

That is, the on-sky localization is given by: 
- right ascension (RA)
- declination (Dec)
- maximum of the real-time elliptical semi-minor (`pos_error_semiminor_deg_95`) and semi-major (`pos_error_semimajor_deg_95`)
- the UTC timestamp, **corrected** for dispersion down to the bottom of the CHIME band (400 MHz)
- the astronomical coordinate system, UTC-FK5-TOPO

This localization region represents the **95%** confidence region. For more details of the localization robustness, please see (**ARXIV LINK**) for a discussion of calibration with the technique of pulsar analogues.

### 8.1 Notes on Comparing with `<What>`

(1) The `<ISOTime>` reported in `<WhereWhen>` is equivalent to the `timestamp_utc` given as a `<Param>` in `<What>`.
(2) The `<Error2Radius>` is equivalent to `max(pos_error_semiminor_deg_95, pos_error_semimajor_deg_95)`.

In [12]:
# Print the entire <WhereWhen> section to standard output

wherewhen_section = voevent_string[
    voevent_string.find("<WhereWhen>"):voevent_string.find("</WhereWhen>") + len("</WhereWhen>")
]
print(wherewhen_section)

<WhereWhen>
    <ObsDataLocation>
      <ObservatoryLocation id="CHIME lives at Dominion Radio Astrophysical Observatory (DRAO)"/>
      <ObservationLocation>
        <AstroCoordSystem id="UTC-FK5-TOPO"/>
        <AstroCoords coord_system_id="UTC-FK5-TOPO">
          <Time unit="s">
            <TimeInstant>
              <ISOTime>2021-08-19T21:50:28.900992</ISOTime>
            </TimeInstant>
          </Time>
          <Position2D unit="deg">
            <Name1>RA</Name1>
            <Name2>Dec</Name2>
            <Value2>
              <C1>0</C1>
              <C2>0</C2>
            </Value2>
            <Error2Radius>0.11850000000000001</Error2Radius>
          </Position2D>
        </AstroCoords>
      </ObservationLocation>
    </ObsDataLocation>
  </WhereWhen>


In [13]:
def get_wherewhen(voevent):
    """
    Return the localization region of a CHIME/FRB VOEvent.
    
    Print the result to the screen and return as a tuple
    with format (RA, Dec, error_radius).
    """
    # The structure of this section contains many nested layers
    
    # Get the coordinates layer
    astro_coords = voevent.WhereWhen.ObsDataLocation.ObservationLocation.AstroCoords
    
    # Get the coordinate system
    system = astro_coords.attrib["coord_system_id"]
    
    # Get the detection timestamp
    timestamp_utc = astro_coords.Time.TimeInstant.ISOTime
    
    # Get the sky position
    position2d = astro_coords.Position2D
    
    # Note: RA is always the C1 value
    ra = position2d.Value2.C1
    
    # Note: Dec is always the C2 value
    dec = position2d.Value2.C2
    
    # Get the uncertainty/error/localization region
    error_radius = position2d.Error2Radius
    
    # Format as a tuple
    region = (ra, dec, error_radius)
    
    print(f"Sky location:\n--> (RA, dec) = ({ra}, {dec}) +/- {error_radius}\n")
    print(f"Detection time [UTC]:\n--> {timestamp_utc}\n")
    print(f"Astronomical coordinate system:\n--> {system}\n")
    return region, timestamp_utc, system

In [14]:
get_wherewhen(voevent)

Sky location:
--> (RA, dec) = (0, 0) +/- 0.11850000000000001

Detection time [UTC]:
--> 2021-08-19T21:50:28.900992

Astronomical coordinate system:
--> UTC-FK5-TOPO



((0, 0, 0.11850000000000001), '2021-08-19T21:50:28.900992', 'UTC-FK5-TOPO')

---
## 9. Get `<How>` Content

The `<How>` section is only lightly used in CHIME/FRB VOEvents, in which one can find a link to the CHIME/FRB public webpage.

Some VOEvent authors (aLIGO/Virgo, for example) include links (URLs, URIs) to analysis products that can be accesses or downloaded upon receipt of the VOEvent. In the future, it is likely that users of the Service will be able to do the same with certain analysis products from CHIME/FRB.

In [15]:
# Print the entire <How> section to standard output

how_section = voevent_string[
    voevent_string.find("<How>"):voevent_string.find("</How>") + len("</How>")
]
print(how_section)

<How>
    <Description>Real-time pipeline topocentric time-of-arrival is corrected for dispersion to bottom of CHIME band (400 MHz)</Description>
    <Reference uri="https://www.chime-frb.ca"/>
  </How>


In [16]:
# Get the public webpage URL
public_webpage = voevent.How.Reference.attrib["uri"]
print(f"The CHIME/FRB public webpage:\n--> {public_webpage}")

The CHIME/FRB public webpage:
--> https://www.chime-frb.ca


---
## 10. Get `<Why>` Content

Here we will instead work with a `subsequent` CHIME/FRB VOEvent to demonstrate details of the `<Why>` section that are not used in `detection` VOEvents.

In [17]:
# Load the subsequent VOEvent
sub_filename = cwd + "../static/FRB-SUBSEQUENT-2021-08-22-23:16:06.571791UTC+0000_f4a33adeaa19.xml"

with open(sub_filename, "rb") as f:
    # Load VOEvent XML from file
    sub_voevent = vp.load(f)
    # Dump VOEvent XML to a string variable
    sub_voevent_string = vp.prettystr(sub_voevent)

# Print the entire <Why> section to standard output

why_section = sub_voevent_string[
    sub_voevent_string.find("<Why"):sub_voevent_string.find("</Why>") + len("</Why>")
]
print(why_section)

<Why importance="0.9880000000000001">
    <Inference probability="0.9998999834060661" relation="Known source association probability">
      <Name>60792403</Name>
      <Concept>The probability is a measure of known source association derived from Bayes factor of known sources in (DM, sky-position) space</Concept>
    </Inference>
    <Description>CHIME/FRB VOEvent Service subsequent-type alert from real-time pipeline</Description>
    <Name>60792403</Name>
    <Concept>Importance is a machine learning score from 0-1 separating RFI (0) from an astrophysical signal (1); Probability is a measure of known source association probability</Concept>
  </Why>


### 10.1 The `importance` of a CHIME/FRB VOEvent

Every CHIME/FRB VOEvent comes with a measure of its `importance` as reported in the `<Why>` section. This `importance` value is a machine learning score generated by a classifier that is trained to distinguish two classes using the real-time meta data available at the L2-L3 stage. The score is a real number scaled from 0 to 1, with 0 indicating a "certain" `RFI` event and 10 indicating a "certain" `Astrophysical` event.

Under normal circumstances, the classifier is trained with a training data set that consists of CHIME/FRB events marked as either `RFI` (0) or `Astrophysical` (10). The output of the classifier during operation is then scaled as a gradient across the decision boundary between these two classes, allowing for a real-valued score between 0 and 10. The training data is prepared by a combination of (1) human labelling and (2) automated labelling (followed up with human correction) using the previous release of the classifier. Finally, the score reported in the VOEvent is scaled down to [0, 1] from [0, 10].

These details are summarized in the `<Concept>` field that comes with the `<Why>` section.

In [18]:
# Get the importance value
importance = sub_voevent.Why.attrib["importance"]
print(f"Importance value is:\n--> {importance}")

Importance value is:
--> 0.9880000000000001


### 10.2 The `probability` of a CHIME/FRB VOEvent

In [19]:
# Get the known source name
known_source_name = vp.get_grouped_params(sub_voevent)["event parameters"]["known_source_name"]["value"]

# Get the probability value
probability = sub_voevent.Why.Inference.attrib["probability"]
print(f"Probability of association with source named '{known_source_name}' is {probability}.")

# Get a description of the importance and probability value
concept = sub_voevent.Why.Inference.Concept
print(f"{concept}.")

Probability of association with source named '60792403' is 0.9998999834060661.
The probability is a measure of known source association derived from Bayes factor of known sources in (DM, sky-position) space.


---

## 11. Get `<Citations>` Content

Here we again work with the `subsequent` VOEvent. Citations refer to previous VOEvents by their IVORN. The CHIME/FRB VOEvent Service publishes `subsequent` VOEvents with citations to the last known `detection` VOEvent for that source, if it exists. Because some repeating FRBs were identified prior to the deployment of the Service, not all `subsequent` VOEvents can refer in this way.

In this section is the IVORN to be cited, as described above, and a flag to indicate why the citation is made. For `subsequent` VOEvents, this will always be `followup`, to indicate that the VOEvent represent an observation of a previously known FRB.

In [20]:
# Print the entire <Citations> section to standard output

citations_section = sub_voevent_string[
    sub_voevent_string.find("<Citations"):sub_voevent_string.find("</Citations>") + len("</Citations>")
]
print(citations_section)

<Citations>
    <EventIVORN cite="followup"/>
  </Citations>


In [21]:
# Get the citation ivorn
citation_ivorn = sub_voevent.Citations.EventIVORN
print(f"Citation IVORN:\n--> {citation_ivorn}\n")

# Get the citation explanation
explanation = sub_voevent.Citations.EventIVORN.attrib["cite"]
print(f"Citation explanation:\n--> {explanation}\n")

Citation IVORN:
--> 

Citation explanation:
--> followup



---
## 12.  Retraction VOEvents

Detection and subsequent VOEvents may be retracted through a retraction VOEvent. In this special kind of VOEvent, the spatial and temporal coordinates of the event are referenced in the retraction, as well as the VOEvent IVORN.

In [27]:
filename = cwd + "../static/OBS-RETRACTION-2021-09-07-09:49:11.821703UTC+0000_65550c2c8d65.xml"

with open(filename, "rb") as f:
    # Load VOEvent XML from file
    ret_voevent = vp.load(f)
    # Dump VOEvent XML to a string variable
    ret_voevent_string = vp.prettystr(voevent)

### 12.1 Coordinates of the Retracted VOEvent

The same method as used above on the detection and subsequent VOEvents will work here. **Note** that these coordinates correspond to the FRB that is being retracted.

In [28]:
get_wherewhen(ret_voevent)

Sky location:
--> (RA, dec) = (0, 0) +/- 0

Detection time [UTC]:
--> 2021-09-07T09:49:11.821703

Astronomical coordinate system:
--> UTC-FK5-TOPO



((0, 0, 0), '2021-09-07T09:49:11.821703', 'UTC-FK5-TOPO')

### 12.2 IVORN of the Retracted VOEvent

Here we compare the IVORN of (A) the retract**ion** VOEvent, and show how it differs from (B) the IVORN of the retract**ed** VOEvent. In (A), the IVORN is found in the usual place, the `<VOEvent>` section. In (B) the IVORN is found in the (also usual) place, the `<Citations>` section. 

**Note** that all retraction VOEvents have an IVORN that uses the `OBS` keyword rather than `FRB`, and the role is set to `"utility"` rather than `"observation"`.

In [29]:
# Get the <VOEvent> section as string
ret_voevent_section = ret_voevent_string[
    ret_voevent_string.find("<VOEvent"):ret_voevent_string.find("<Who>")
]
print(ret_voevent_section)

# Get the VOEvent IVORN from the <VOEvent> section
ivorn = ret_voevent.attrib["ivorn"]
print(f"VOEvent IVORN:\n--> {ivorn}\n")

# Get the VOEvent role from the <VOEvent> section
role = ret_voevent.attrib["role"]
print(f"VOEvent role:\n--> {role}\n")

<VOEvent xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.ivoa.net/xml/VOEvent/v2.0 http://www.ivoa.net/xml/VOEvent/VOEvent-v2.0.xsd" version="2.0" role="utility" ivorn="ivo://ca.chimenet.frb/OBS-RETRACTION-#2021-09-07-09:49:11.821703UTC+0000_65550c2c8d65">
  
VOEvent IVORN:
--> ivo://ca.chimenet.frb/OBS-RETRACTION-#2021-09-07-09:49:11.821703UTC+0000_65550c2c8d65

VOEvent role:
--> utility



In [30]:
# Get the citation ivorn
citation_ivorn = ret_voevent.Citations.EventIVORN
print(f"Citation IVORN:\n--> {citation_ivorn}\n")

# Get the citation explanation
explanation = ret_voevent.Citations.EventIVORN.attrib["cite"]
print(f"Citation explanation:\n--> {explanation}\n")

Citation IVORN:
--> ivo://ca.chimenet.frb/FRB-DETECTION-#2021-09-07-09:48:59.922483UTC+0000_daa68ff4aeaf

Citation explanation:
--> retraction



---
## 13. Conclusion

In this notebook we demonstrated how to extract all useful components of CHIME/FRB VOEvents, focusing on `detection` and `susbequent` VOEvents. New subscribers should develop their event handlers using this as reference.

## 14. Future Work

As the Service matures, other VOEvent types will become available from CHIME/FRB, including several different `update` alerts.

These post real-time VOEvents will have their own content, but the methods exemplified here will apply just the same. Nonetheless, the user can expect this tutorial to evolve accordingly.